In [178]:
# Pasos previos: obtener credenciales en https://www.buenosaires.gob.ar/desarrollourbano/transporte/apitransporte
# Toda la documentacion esta disponible en: https://www.buenosaires.gob.ar/desarrollourbano/transporte/apitransporte/api-doc

In [179]:
import time
import sys
import pandas as pd
import numpy as np # type: ignore
import os
import urllib.request
import re
import requests # type: ignore
import json
from pandas import json_normalize # type: ignore
from datetime import date, datetime
from pandas import Timestamp # type: ignore
import seaborn as sns # type: ignore
import matplotlib.pyplot as plt # type: ignore
from matplotlib.pyplot import figure, show, suptitle # type: ignore
def arreglar_json(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    entity_data = data.get("_entity", [])
    
    with open(json_file, 'w') as f:
        json.dump(entity_data, f, indent=4)

def json2csv(json_file_path, csv_file_path):
    """
    Convierte un archivo JSON en un archivo CSV con columnas seleccionadas.

    Args:
        json_file_path (str): La ruta del archivo JSON.
        csv_file_path (str): La ruta del archivo CSV de salida.
    """
    try:
        # Cargar el JSON desde el archivo
        with open(json_file_path, "r") as f:
            json_data = json.load(f)

        # Convertir el JSON en un DataFrame
        df = pd.json_normalize(json_data)

        # Seleccionar las columnas relevantes
        selected_columns = ['_id', '_vehicle._position._latitude', '_vehicle._position._longitude', 
                            '_vehicle._position._speed', '_vehicle._stop_id', '_vehicle._timestamp', 
                            '_vehicle._trip._route_id', '_vehicle._trip._trip_id']
        df_selected = df[selected_columns]

        # Guardar el DataFrame como un archivo CSV
        df_selected.to_csv(csv_file_path, index=False)

        print(f'Se ha guardado el archivo CSV como "{csv_file_path}"')
    except Exception as e:
        print("Error:", e)

def guardar_json_desde_url(url, file_path):
    """
    Guarda un JSON desde una URL en un archivo especificado.

    Args:
        url (str): La URL desde donde obtener el JSON.
        file_path (str): La ruta del archivo donde guardar el JSON.

    Returns:
        str: El nombre del archivo guardado si la operación se realizó con éxito, None en caso contrario.
    """
    try:
        # Realizamos la solicitud HTTP
        response = requests.get(url)

        # Verificamos si la solicitud fue exitosa
        if response.status_code == 200:
            # Cargamos el JSON desde la respuesta
            data = response.json()

            # Guardamos el JSON en el archivo especificado
            with open(file_path, "w") as outfile:
                json.dump(data, outfile, indent=4)
            
            return file_path
        else:
            print("Error al realizar la solicitud:", response.status_code)
            return None
    except Exception as e:
        print("Error:", e)
        return None


In [180]:
#configuracion para poder ver todas las columnas del dataset
pd.set_option('display.max_columns', 500)

## Vamos a la API de Transporte:

In [181]:
# Completa con tus credenciales obtenidas de la API de Transporte:
client_id = 'TOKEN_PARA_API'
client_secret = 'TOKEN_PARA_API'

In [1]:
# Request de GPS de linea 145
link_api_generico= f"https://apitransporte.buenosaires.gob.ar/colectivos/vehiclePositionsSimple?client_id={client_id}&client_secret={client_secret}"
link_api_145 = f"https://apitransporte.buenosaires.gob.ar/colectivos/vehiclePositions?client_id={client_id}&client_secret={client_secret}&json=1&agency_id=516"

NameError: name 'client_id' is not defined

In [186]:
# Coordenadas del colectivo por su ID
# Seleccionar solo las columnas 'latitude' y 'longitude'
# coordenadas = data_by_id[['latitude', 'longitude']] 
# latitud = coordenadas['latitude'].iloc[0]
# longitude = coordenadas['longitude'].iloc[0]
# print(f"Las coordenadas del colectivo con id 4634 son: {latitud}, {longitude}")

In [187]:
###En el caso de que no llegue a funcionar. El csv pueden obtenerlo en el github:
##arr_and_dep = pd.read_csv("https://github.com/datosgcba/taller-api-transporte/blob/master/oba.csv", error_bad_lines=False)

### Buscar ID de una línea:

In [195]:
# Vamos a usar el csv de paradas de colectivos que está en BA Data (https://data.buenosaires.gob.ar/)
datacsv=pd.read_csv('http://cdn.buenosaires.gob.ar/datosabiertos/datasets/colectivos/paradas-de-colectivo.csv')

# Uso algunos csv ya cargados (tambien sacados de la API BA Data)

In [196]:
# Los primeros elementos del dataframe data se ven asi
datacsv.head()

,stop_id,stop_code,stop_name,stop_lat,stop_lon,direction_id,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type
0,201,201,1520 DEFENSA,-34.625663,-58.371042,1,108,26,22A,JNAMBA022,Retiro - Cno. Centenario y Lavalleja,3
1,2072,2072,722 REGIMIENTO DE PATRICIOS AV.,-34.636743,-58.369765,1,108,26,22A,JNAMBA022,Retiro - Cno. Centenario y Lavalleja,3
2,2079,2079,324 REGIMIENTO DE PATRICIOS AV.,-34.632942,-58.370332,1,108,26,22A,JNAMBA022,Retiro - Cno. Centenario y Lavalleja,3
3,2084,2084,16 REGIMIENTO DE PATRICIOS AV.,-34.629952,-58.370600,1,108,26,22A,JNAMBA022,Retiro - Cno. Centenario y Lavalleja,3
4,20350,20350,1776 MONTES DE OCA MANUEL AV.,-34.647807,-58.372853,1,108,26,22A,JNAMBA022,Retiro - Cno. Centenario y Lavalleja,3


In [197]:
# Podes usar el JSON que deje ya cargado para obtener los id de un colectivo
data = pd.read_json("C:\\Users\\Mateo\\Desktop\\test\\api-transporteBA\\data\\lineas_colectivo.json")

# Asi se ve el json
data

,route_id,latitude,longitude,speed,timestamp,id,direction,agency_name,agency_id,route_short_name,tip_id,trip_headsign
0,1465,-34.68544,-58.684140,12.500000,2024-04-18 23:14:46,1822,1,MICROOMNIBUS SAAVEDRA S.A.T.A.C.I.,82,253A,95269-1,Expreso x Baradero - Olimpo y C. de Cintura
1,1467,-34.65992,-58.666240,0.000000,2024-04-18 23:14:46,1838,1,MICROOMNIBUS SAAVEDRA S.A.T.A.C.I.,82,321A,95366-1,a Chacarita x Camarones
2,1468,-34.64760,-58.607220,0.000000,2024-04-18 23:14:46,1840,0,MICROOMNIBUS SAAVEDRA S.A.T.A.C.I.,82,153A,95454-1,a Cañuelas
3,1464,-34.68401,-58.682495,12.222222,2024-04-18 23:14:44,1845,0,MICROOMNIBUS SAAVEDRA S.A.T.A.C.I.,82,253A,95222-1,a Liniers
4,1468,-34.66011,-58.622820,0.555555,2024-04-18 23:14:44,1854,0,MICROOMNIBUS SAAVEDRA S.A.T.A.C.I.,82,153A,95452-1,a Cañuelas
...,...,...,...,...,...,...,...,...,...,...,...,...
7882,1090,-34.50753,-58.581795,0.000000,2024-04-18 23:00:46,69712,0,MICRO OMNIBUS GENERAL SAN MARTIN SAC,141,333,71108-1,Palomar x Av. de Mayo - IDA
7883,248,-34.62370,-58.444706,0.000000,2024-04-18 23:14:46,69753,0,LA VECINAL DE MATANZA SACI. DE MICROOMNIBUS,76,180I,19834-1,a EL PATO - x Ruta 36
7884,1821,-34.48770,-58.595055,25.277777,2024-04-18 23:14:46,69756,1,MICRO OMNIBUS NORTE S.A.,69,60H,120387-1,a Constitución
7885,1361,-34.61829,-58.409280,4.722222,2024-04-18 23:14:46,69767,1,AZUL S.A. DE TRANSPORTE AUTOMOTOR,49,41A,88500-1,Ramal 54 - VUELTA


In [199]:
# Ahora tenes que usando el dataframe data filtar por la linea que quieras en el JSON
linea_colectivo = "99A"

# Filtrar el DataFrame por la línea de colectivo específica
filtro_linea = data[data.route_short_name == linea_colectivo]

# Muestro el dataframe, solo tenes que copiar algun id que quieras y eso lo pasas al GPS del main
filtro_linea

,route_id,latitude,longitude,speed,timestamp,id,direction,agency_name,agency_id,route_short_name,tip_id,trip_headsign
685,1523,-34.59770,-58.420060,2.500000,2024-04-18 23:14:46,5050,1,COLECTIVEROS UNIDOS S.A.,29,99A,98633-1,a Pte. La Noria
687,1522,-34.60066,-58.386900,0.000000,2024-04-18 23:14:46,5053,0,COLECTIVEROS UNIDOS S.A.,29,99A,98421-1,a Est. Lomas
688,1523,-34.60528,-58.438840,4.444444,2024-04-18 23:14:46,5057,1,COLECTIVEROS UNIDOS S.A.,29,99A,98632-1,a Pte. La Noria
690,1522,-34.61694,-58.462080,8.055555,2024-04-18 23:14:48,5059,0,COLECTIVEROS UNIDOS S.A.,29,99A,98429-1,a Est. Lomas
691,1522,-34.60059,-58.406746,0.000000,2024-04-18 23:14:48,5060,0,COLECTIVEROS UNIDOS S.A.,29,99A,98425-1,a Est. Lomas
692,1522,-34.62715,-58.491160,0.000000,2024-04-18 23:14:44,5062,0,COLECTIVEROS UNIDOS S.A.,29,99A,98431-1,a Est. Lomas
693,1523,-34.63237,-58.501500,11.944444,2024-04-18 23:14:46,5064,1,COLECTIVEROS UNIDOS S.A.,29,99A,98622-1,a Pte. La Noria
694,1523,-34.61768,-58.468914,7.222222,2024-04-18 23:14:46,5065,1,COLECTIVEROS UNIDOS S.A.,29,99A,98630-1,a Pte. La Noria
695,1523,-34.59982,-58.394226,0.000000,2024-04-18 23:14:44,5067,1,COLECTIVEROS UNIDOS S.A.,29,99A,98635-1,a Pte. La Noria
696,1523,-34.59932,-58.383854,0.000000,2024-04-18 23:14:48,5070,1,COLECTIVEROS UNIDOS S.A.,29,99A,98637-1,a Pte. La Noria
